# Edit Training Schedules to Recurr Weekly

This robot notebook deletes a set of members from the club's TeamApp. This is used to delete all members that are not part of the new season.

Demo of bulk editing for recurring schedules: https://youtu.be/Ur2b5QcnmCo

[![https://www.youtube.com/watch?v=Ur2b5QcnmCo](https://img.youtube.com/vi/Ur2b5QcnmCo/0.jpg)](https://www.youtube.com/watch?v=Ur2b5QcnmCo)



In [181]:
import time

# Download geckodriver (https://github.com/mozilla/geckodriver/releases) and put it in path
# Salenium webdriver: https://www.selenium.dev/documentation/overview/
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By

In [131]:
def report_element(e, i=""):
    print(i, e)
    print(i, e.text)
    print(i, e.get_attribute("class"))
    print(i, e.id)
    print(i, e.tag_name)

def report_elements(browser, search_string):    # get all the three dots edit options
    print(search_string)
    for i, button in enumerate(browser.find_elements(By.XPATH, search_string)):
        report_element(button, i)
        print()

## Create Virtual Browser

Create a firefox instance using an existing profile that MUST be logged into TeamApp.

Set the following variables and in particular `FIREFOX_PROFILE_PATH` to the existing Firefox profile that is already logged into the club's TeamApp.

In [2]:
# URL of the club's TeamApp
URL_TEAMAPP = "https://brunswickmagicbasketball.teamapp.com"

# Folder with the firefox profile to be used
FIREFOX_PROFILE_PATH = '/home/ssardina/.mozilla/firefox/88weweww.ssardina'

GECKODRIVER_PATH = '/home/ssardina/bin/geckodriver'

In [ ]:
URL_MODULE = f"{URL_TEAMAPP}/events?_list=v1&team_id=all"

# firefox profile of user already logged in teamapp

options = Options()
# options.headless = True
options.page_load_strategy = 'eager'
options.set_preference('profile', FIREFOX_PROFILE_PATH)

# https://github.com/SeleniumHQ/selenium/issues/11028
# use an existing firefox profile (must be logged in teamapps already!)
options.add_argument("-profile")
options.add_argument(FIREFOX_PROFILE_PATH)

service = None
# service = Service(GECKODRIVER_PATH)

if service is not None:
    browser = webdriver.Firefox(service=service, options=options)
else:
    browser = webdriver.Firefox(options=options)
# browser.maximize_window()

browser.get(URL_MODULE)

# browser.quit()


In case you want to close it:

In [ ]:
browser.quit()

# Test recurr one schedule

We start by retrieving the member card:

In [161]:
# get to the All Schedule Entries
browser.get(URL_MODULE)

### 1. Go to the team schedule

#### Option 1: Directly click three dots of the team (PROBLEM!)

NOTE: when I do this option, the `Team` field comes empty! :_()

In [152]:
# get all the three dots edit options
buttons_teams = browser.find_elements(By.XPATH, '//i[contains(@class, "mdi-dots-vertical")]')

for i, button in enumerate(buttons_teams):
    report_element(button, i)
    print()

In [106]:
TEAM_NO = 1

# Open the advanced options for a particular team
button = buttons_teams[TEAM_NO]
button.click()

time.sleep(2)

# now click EDIT
buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Edit') and contains(@class, 'action-list-item-title')]")
buttons[0].click()


#### Option 2: Select a team, then edit (preferred)

First enter into the team schedule:

In [176]:
# the team number you want to edit
TEAM_NO = 1

# get the button of all the training schedules in the screen (listed with names "U08 Mixed Gold Training")
buttons = browser.find_elements(By.XPATH, r"//*[contains(text(),'Training') and @class='v-list-item__title']")
buttons[1].click()
print("Schedule to edit:" , buttons[1].text)

Schedule to edit:  U10 Girls Black Training


Next click the three dot options top right and then EDIT:

In [164]:
# click options three dots button (top-right)
buttons = browser.find_elements(By.XPATH, '//*[contains(text(), "more_vert")]')
buttons[0].click()

# now EDIT!
buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Edit') and @class='v-list-item__title']")
# report_elements(browser, "//*[contains(text(),'Edit')]")
buttons[0].click()

### 2. Now edit the open schedule to recurr

Now we edit the schedule to recurr.

In [165]:
# select Every week in Recurring
buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Every week')]")
buttons[0].click()

IndexError: list index out of range

Next click to open the `Recurring Ends` date dialog to get the calendar:

In [157]:
buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Recurring Ends')]")
buttons[0].click()

Calendar should come now. We need to navigate to the month and then select the day.

In [159]:

# one month before
buttons = browser.find_elements(By.XPATH, '//i[contains(@class, "mdi-chevron-left")]')
buttons[0].click()

time.sleep(0.5)

# select 21
buttons = browser.find_elements(By.XPATH, "//*[text() = '21']")
buttons[0].click()

time.sleep(0.5)

# select OK
buttons = browser.find_elements(By.XPATH, "//*[contains(text(), 'OK') and contains(@class, 'v-btn__content')]")
buttons[0].click()

Finally, click `SAVE` the schedule:

In [160]:
buttons = browser.find_elements(By.XPATH, "//*[contains(text(), 'Save') and contains(@class, 'v-btn__content')]")
buttons[0].click()

# Bulk edit schedule to recurr

In [180]:
# get to the All Schedule Entries
browser.get(URL_MODULE)

NO_TEAMS = len(browser.find_elements(By.XPATH, r"//*[contains(text(),'Training') and @class='v-list-item__title']"))

for i in range(5):
    browser.get(URL_MODULE)

    time.sleep(3)

    # get the button of all the training schedules in the screen (listed with names "U08 Mixed Gold Training")
    buttons = browser.find_elements(By.XPATH, r"//*[contains(text(),'Training') and @class='v-list-item__title']")
    schedule_title = buttons[i].text
    print("Schedule to edit:", schedule_title)
    buttons[i].click()

    time.sleep(2)

    # click options three dots button (top-right)
    buttons = browser.find_elements(By.XPATH, '//*[contains(text(), "more_vert")]')
    buttons[0].click()

    # now EDIT schedule!
    buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Edit') and @class='v-list-item__title']")
    # report_elements(browser, "//*[contains(text(),'Edit')]")
    buttons[0].click()

    time.sleep(2)
    # select Every week in Recurring (may fail if already recurrent!)
    try:
        buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Every week')]")
        buttons[0].click()
    except IndexError:
        print("\t Team already recurrent!", schedule_title)
        continue

    # click recurring ends to open calendar
    buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Recurring Ends')]")
    buttons[0].click()


    # one month before to september 2024
    buttons = browser.find_elements(By.XPATH, '//i[contains(@class, "mdi-chevron-left")]')
    buttons[0].click()

    time.sleep(0.5)

    # select & click day 21
    buttons = browser.find_elements(By.XPATH, "//*[text() = '21']")
    buttons[0].click()

    time.sleep(0.5)

    # click OK
    buttons = browser.find_elements(By.XPATH, "//*[contains(text(), 'OK') and contains(@class, 'v-btn__content')]")
    buttons[0].click()

    # click SAVE
    buttons = browser.find_elements(By.XPATH, "//*[contains(text(), 'Save') and contains(@class, 'v-btn__content')]")
    buttons[0].click()

    print("\t Team saved!", schedule_title)

    # wait 3 seconds for next schedule
    time.sleep(3)


Schedule to edit: U08 Mixed Gold Training
	 Team already recurrent! U08 Mixed Gold Training
Schedule to edit: U10 Girls Black Training
Schedule to edit: U10 Girls Gold Training
Schedule to edit: U12 Girls Purple Training
Schedule to edit: U10 Boys Purple Training
